# MCP + LangGraph Hands-On Tutorial

- Author: [Teddy Notes](https://youtube.com/c/teddynote)
- Lecture: [Fastcampus RAG trick notes](https://fastcampus.co.kr/data_online_teddy)

**References**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## configure

Refer to the installation instructions below to install `uv`.

**How to install `uv`**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

Install **dependencies**

```bash
uv pip install -r requirements.txt
```

Gets the environment variables.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## MultiServerMCPClient

Run `mcp_server_remote.py` in advance. Open a terminal with the virtual environment activated and run the server.

> Command
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

Create and terminate a temporary Session connection using `async with`

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from utils import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

async with MultiServerMCPClient(
    {
        "weather": {
            # Must match the server's port (port 8005)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(
        agent, {"messages": "What's the weather like in Seoul?"}
    )

You might notice that you can't access the tool because the session is closed.

In [ ]:
await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})

Now let's change that to accessing the tool while maintaining an Async Session.

In [ ]:
# 1. Create client
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. Explicitly initialize connection (this part is necessary)
# Initialize
await client.__aenter__()

# Now tools are loaded
print(client.get_tools())  # Tools are displayed

Create an agent with langgraph(`create_react_agent`).

In [5]:
# Create agent
agent = create_react_agent(model, client.get_tools())

Run the graph to see the results.

In [ ]:
await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})

## Stdio method

The Stdio method is intended for use in a local environment.

- Use standard input/output for communication

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# Set up StdIO server parameters
# - command: Path to Python interpreter
# - args: MCP server script to execute
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["mcp_server_local.py"],
)

# Use StdIO client to communicate with the server
async with stdio_client(server_params) as (read, write):
    # Create client session
    async with ClientSession(read, write) as session:
        # Initialize connection
        await session.initialize()

        # Load MCP tools
        tools = await load_mcp_tools(session)
        print(tools)

        # Create agent
        agent = create_react_agent(model, tools)

        # Stream agent responses
        await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})

## Use MCP server with RAG deployed

- File: `mcp_server_rag.py`

Use the `mcp_server_rag.py` file that we built with langchain in advance.

It uses stdio communication to get information about the tools, where it gets the `retriever` tool, which is the tool defined in `mcp_server_rag.py`. This file **doesn't** need to be running on the server beforehand.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from utils import astream_graph

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# Set up StdIO server parameters for the RAG server
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["./mcp_server_rag.py"],
)

# Use StdIO client to communicate with the RAG server
async with stdio_client(server_params) as (read, write):
    # Create client session
    async with ClientSession(read, write) as session:
        # Initialize connection
        await session.initialize()

        # Load MCP tools (in this case, the retriever tool)
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)

        # Stream agent responses
        await astream_graph(
            agent,
            {
                "messages": "Search for the name of the generative AI developed by Samsung Electronics"
            },
        )

## Use a mix of SSE and Stdio methods

- File: `mcp_server_rag.py` communicates over Stdio
- `langchain-dev-docs` communicates via SSE

Use a mix of SSE and Stdio methods.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 1. Create multi-server MCP client
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "./.venv/bin/python",
            # Update with the absolute path to mcp_server_rag.py file
            "args": ["./mcp_server_rag.py"],
            # Communicate via stdio (using standard input/output)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # Make sure the SSE server is running
            "url": "https://teddynote.io/mcp/langchain/sse",
            # Communicate via SSE (Server-Sent Events)
            "transport": "sse",
        },
    }
)


# 2. Initialize connection explicitly through async context manager
await client.__aenter__()

Create an agent using `create_react_agent` in langgraph.

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in English."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

Use the `retriever` tool defined in `mcp_server_rag.py` that you built to perform the search.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "Use the `retriever` tool to search for the name of the generative AI developed by Samsung Electronics"
    },
    config=config,
)

This time, we'll use the `langchain-dev-docs` tool to perform the search.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "Please tell me about the definition of self-rag by referring to the langchain-dev-docs"
    },
    config=config,
)

Use `MemorySaver` to maintain short-term memory, so multi-turn conversations are possible.

In [ ]:
await astream_graph(
    agent,
    {"messages": "Summarize the previous content in bullet points"},
    config=config,
)

## LangChain-integrated tools + MCP tools

Here we confirm that tools integrated into LangChain can be used in conjunction with existing MCP-only tools.

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the Tavily search tool (news type, news from the last 3 days)
tavily = TavilySearchResults(max_results=3, topic="news", days=3)

# Use it together with existing MCP tools
tools = client.get_tools() + [tavily]

Create an agent using `create_react_agent` in langgraph.

In [16]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in English."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

Perform a search using the newly added `tavily` tool.

In [ ]:
await astream_graph(
    agent, {"messages": "Tell me about today's news for me"}, config=config
)

You can see that the `retriever` tool is working smoothly.

In [ ]:
await astream_graph(
    agent,
    {
        "messages": "Use the `retriever` tool to search for the name of the generative AI developed by Samsung Electronics"
    },
    config=config,
)

## Smithery MCP Server

- Link: https://smithery.ai/

List of tools used:

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - MCP server providing tools for dynamic and reflective problem-solving through structured thinking processes
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - Run terminal commands and manage files with various editing capabilities. Coding, shell and terminal, task automation

**Note**

- When importing tools provided by smithery in JSON format, you must set `"transport": "stdio"` as shown in the example below.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Initialize LLM model
model = ChatAnthropic(model="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000)

# 1. Create client
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "your_smithery_api_key",
            ],
            "transport": "stdio",  # Add communication using stdio method
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "your_smithery_api_key",
            ],
            "transport": "stdio",  # Add communication using stdio method
        },
        "document-retriever": {
            "command": "./.venv/bin/python",
            # Update with the absolute path to the mcp_server_rag.py file
            "args": ["./mcp_server_rag.py"],
            # Communication using stdio (standard input/output)
            "transport": "stdio",
        },
    }
)


# 2. Explicitly initialize connection
await client.__aenter__()

Create an agent using `create_react_agent` in langgraph.

In [23]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

# Set up configuration
config = RunnableConfig(recursion_limit=30, thread_id=3)

# Create agent
agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [ ]:
await astream_graph(
    agent,
    {
        "messages": "Draw the folder structure including the current path as a tree. However, exclude the .venv folder from the output."
    },
    config=config,
)

We'll use the `Sequential Thinking` tool to see if we can accomplish a relatively complex task.

In [ ]:
await astream_graph(
    agent,
    {
        "messages": (
            "Use the `retriever` tool to search for information about generative AI developed by Samsung Electronics, "
            "and then use the `Sequential Thinking` tool to write a report."
        )
    },
    config=config,
)